# Tugas 2 : Preprocessing  & TfIdf

In [1]:
from google.colab import files

# Upload file dari komputer
uploaded = files.upload()

Saving data_berita.csv to data_berita.csv


In [3]:
import pandas as pd
import io

# Membaca file CSV yang diunggah dan menyimpannya ke dalam DataFrame dengan variabel df
df = pd.read_csv(io.BytesIO(uploaded['data_berita.csv']))

df.head(5) # Menampilkan 5 baris pertama dari DataFrame

,judul,tanggal,isi,kategori
0,Badan Kurus Tak Berarti Bebas dari Kolesterol ...,"Kamis, 05 Sep 2024 11:03 WIB",Jakarta - Tak sedikit yang beranggapan bahwa o...,Kesehatan
1,Budaya Kerja Toksik di Jepang Picu 54 Karyawan...,"Kamis, 05 Sep 2024 10:38 WIB",Jakarta -\n \n Istilah 'bekerja ...,Kesehatan
2,Menyoal Serangan Jantung yang Dialami Faisal B...,"Kamis, 05 Sep 2024 10:03 WIB",Jakarta - Faisal Basri meninggal dunia. Ekonom...,Kesehatan
3,Nyeri Kronis yang Bikin Paus Fransiskus Tak Bi...,"Kamis, 05 Sep 2024 09:37 WIB",Jakarta - Pemimpin tertinggi Gereja Katolik se...,Kesehatan
4,"Perundungan PPDS Tak Cuma Terjadi di Undip, Be...","Kamis, 05 Sep 2024 08:35 WIB",Jakarta - Aksi bullying atau perundungan di ma...,Kesehatan


In [4]:
# Melihat jumlah baris dan kolom
print(f"Jumlah baris: {df.shape[0]}\nJumlah kolom: {df.shape[1]}")

Jumlah baris: 100
Jumlah kolom: 4


In [6]:
df_review = df[['judul',	'tanggal',	'isi',	'kategori']] # Memilih atribut/kolom penting
df_review = df_review.dropna(subset=['isi']) # Menghapus missing value pada atribut/kolom content
df_review.head(5)

,judul,tanggal,isi,kategori
0,Badan Kurus Tak Berarti Bebas dari Kolesterol ...,"Kamis, 05 Sep 2024 11:03 WIB",Jakarta - Tak sedikit yang beranggapan bahwa o...,Kesehatan
1,Budaya Kerja Toksik di Jepang Picu 54 Karyawan...,"Kamis, 05 Sep 2024 10:38 WIB",Jakarta -\n \n Istilah 'bekerja ...,Kesehatan
2,Menyoal Serangan Jantung yang Dialami Faisal B...,"Kamis, 05 Sep 2024 10:03 WIB",Jakarta - Faisal Basri meninggal dunia. Ekonom...,Kesehatan
3,Nyeri Kronis yang Bikin Paus Fransiskus Tak Bi...,"Kamis, 05 Sep 2024 09:37 WIB",Jakarta - Pemimpin tertinggi Gereja Katolik se...,Kesehatan
4,"Perundungan PPDS Tak Cuma Terjadi di Undip, Be...","Kamis, 05 Sep 2024 08:35 WIB",Jakarta - Aksi bullying atau perundungan di ma...,Kesehatan


In [7]:
# Melihat jumlah baris dan kolom
print(f"Jumlah baris: {df_review.shape[0]}\nJumlah kolom: {df_review.shape[1]}")

Jumlah baris: 100
Jumlah kolom: 4


***

#<b>2. Text Preprocessing</b>

##<b>2.1 Text Cleaning</b>

In [31]:
import re

def text_cleaning(text):
    """
    Membersihkan teks dengan menghapus mention, hashtag, retweet prefix, URL, emotikon,
    karakter non-alfanumerik, HTML tags, dan menormalkan spasi. Termasuk juga menangani
    singkatan dan akronim dengan titik.

    Parameters:
    text (str): Teks yang akan dibersihkan.

    Returns:
    str: Teks yang telah dibersihkan.
    """
    # Menghapus URL
    text = re.sub(r'https?://\S+|www\.\S+', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()  # Normalisasi spasi

    # Menghapus HTML tags
    text = re.sub(r'<[^>]+>', '', text)

    # Menghapus &amp
    text = re.sub(r'&amp;', '', text)

    # Menambahkan spasi di antara huruf dan titik
    text = re.sub(r'(?<=\w)\.(?=\w)', ' ', text)

    # Mengganti karakter non-breaking space dengan spasi biasa
    text = text.replace('\xa0', ' ')

    # Menghapus emotikon
    EMOJI_PATTERN = re.compile(
        "["
        "\U0001F1E0-\U0001F1FF"  # Flags
        "\U0001F300-\U0001F5FF"  # Symbols & pictographs
        "\U0001F600-\U0001F64F"  # Emoticons
        "\U0001F680-\U0001F6FF"  # Transport & map symbols
        "\U0001F700-\U0001F77F"  # Alchemical symbols
        "\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
        "\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
        "\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        "\U0001FA00-\U0001FA6F"  # Chess Symbols
        "\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
        "\U00002702-\U000027B0"  # Dingbats
        "]"
    )
    text = re.sub(EMOJI_PATTERN, '', text)

    # Menghapus mention (misalnya, @username)
    text = re.sub(r'@[A-Za-z0-9_]+', '', text)

    # Menghapus hashtag (misalnya, #hashtag)
    text = re.sub(r'#\w+', '', text)

    # Menghapus prefix "RT" yang sering digunakan di Twitter untuk retweet
    text = re.sub(r'RT[\s]+', '', text)

    # Menghapus angka dan karakter non-alfanumerik selain spasi
    text = re.sub(r'[0-9]', '', text)  # Menghapus angka
    text = re.sub(r'[^A-Za-z ]', '', text)  # Menghapus karakter non-alfanumerik lainnya

    # Menghapus baris baru dan carriage return
    text = re.sub(r'[\n\r]', '', text)

    # Normalisasi spasi ganda menjadi satu spasi dan menghapus spasi ekstra di awal dan akhir
    text = re.sub(r'\s+', ' ', text).strip()

    return text


In [32]:
# Membuat salinan DataFrame
df_txt = df_review.copy()

# Menggunakan fungsi text_cleaning pada kolom 'full_text' dan menampilkan hasilnya di kolom baru 'cleaned_text'
df_txt['cleaned_text'] = df_txt['isi'].apply(lambda x: text_cleaning(x))
df_txt.head(5)

,judul,tanggal,isi,kategori,cleaned_text
0,Badan Kurus Tak Berarti Bebas dari Kolesterol ...,"Kamis, 05 Sep 2024 11:03 WIB",Jakarta - Tak sedikit yang beranggapan bahwa o...,Kesehatan,Jakarta Tak sedikit yang beranggapan bahwa ora...
1,Budaya Kerja Toksik di Jepang Picu 54 Karyawan...,"Kamis, 05 Sep 2024 10:38 WIB",Jakarta -\n \n Istilah 'bekerja ...,Kesehatan,Jakarta Istilah bekerja sampai mati nampaknya ...
2,Menyoal Serangan Jantung yang Dialami Faisal B...,"Kamis, 05 Sep 2024 10:03 WIB",Jakarta - Faisal Basri meninggal dunia. Ekonom...,Kesehatan,Jakarta Faisal Basri meninggal dunia Ekonom Un...
3,Nyeri Kronis yang Bikin Paus Fransiskus Tak Bi...,"Kamis, 05 Sep 2024 09:37 WIB",Jakarta - Pemimpin tertinggi Gereja Katolik se...,Kesehatan,Jakarta Pemimpin tertinggi Gereja Katolik sedu...
4,"Perundungan PPDS Tak Cuma Terjadi di Undip, Be...","Kamis, 05 Sep 2024 08:35 WIB",Jakarta - Aksi bullying atau perundungan di ma...,Kesehatan,Jakarta Aksi bullying atau perundungan di masa...


##<b>2.2 Case Folding & Normalizing</b>

In [33]:
# Daftar penggantian kata untuk normalisasi teks
# Run cell ini sebelum menjalankan fungsi text_normalize
# Tambahkan daftar kata jika ada yang kurang dengan format {' kata_nonformal ':' kata_formal '}
norm_list = {

    'tak': 'tidak',
    'bikin': 'membuat',
    'gak': 'tidak',
    'nggak': 'tidak',
    'udah': 'sudah',
    'ngomong': 'berbicara',
    'dibilang': 'dikatakan',
    'bilang': 'mengatakan',
    'aja': 'saja',
    'kayak': 'seperti',
    'begini': 'seperti ini',
    'begitu': 'seperti itu'

}

In [34]:
def text_normalize(text):
    """
    Menormalkan teks dengan menggantikan singkatan atau bentuk tidak formal dengan bentuk formal,
    serta mengubah semua huruf menjadi huruf kecil (case folding).

    Parameters:
    text (str): Teks yang akan dinormalisasi.

    Returns:
    str: Teks yang telah dinormalisasi.
    """
    # Mengubah semua huruf menjadi huruf kecil (case folding)
    text = text.lower()

    # Mengganti singkatan atau kata tidak formal dengan bentuk formal menggunakan dictionary norm_list
    for key, value in norm_list.items():
        text = text.replace(key, value)

    # Normalisasi spasi ganda menjadi satu spasi dan menghapus spasi ekstra di awal dan akhir
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [35]:
# Menggunakan fungsi text_normalize pada kolom 'cleaned_text' dan menampilkan hasilnya di kolom baru 'norm_text'
df_txt['norm_text'] = df_txt['cleaned_text'].apply(lambda x: text_normalize(x))
df_txt.head(5)

,judul,tanggal,isi,kategori,cleaned_text,norm_text
0,Badan Kurus Tak Berarti Bebas dari Kolesterol ...,"Kamis, 05 Sep 2024 11:03 WIB",Jakarta - Tak sedikit yang beranggapan bahwa o...,Kesehatan,Jakarta Tak sedikit yang beranggapan bahwa ora...,jakarta tidak sedikit yang beranggapan bahwa o...
1,Budaya Kerja Toksik di Jepang Picu 54 Karyawan...,"Kamis, 05 Sep 2024 10:38 WIB",Jakarta -\n \n Istilah 'bekerja ...,Kesehatan,Jakarta Istilah bekerja sampai mati nampaknya ...,jakarta istilah bekerja sampai mati nampaknya ...
2,Menyoal Serangan Jantung yang Dialami Faisal B...,"Kamis, 05 Sep 2024 10:03 WIB",Jakarta - Faisal Basri meninggal dunia. Ekonom...,Kesehatan,Jakarta Faisal Basri meninggal dunia Ekonom Un...,jakarta faisal basri meninggal dunia ekonom un...
3,Nyeri Kronis yang Bikin Paus Fransiskus Tak Bi...,"Kamis, 05 Sep 2024 09:37 WIB",Jakarta - Pemimpin tertinggi Gereja Katolik se...,Kesehatan,Jakarta Pemimpin tertinggi Gereja Katolik sedu...,jakarta pemimpin tertinggi gereja katolik sedu...
4,"Perundungan PPDS Tak Cuma Terjadi di Undip, Be...","Kamis, 05 Sep 2024 08:35 WIB",Jakarta - Aksi bullying atau perundungan di ma...,Kesehatan,Jakarta Aksi bullying atau perundungan di masa...,jakarta aksi bullying atau perundungan di masa...


##<b>2.3 Tokenizing</b>

In [36]:
# Membagi teks pada kolom 'norm_text' menjadi token-token (kata-kata) berdasarkan spasi dan menampilkan hasilnya di kolom baru 'token_text'
df_txt['token_text'] = df_txt['norm_text'].apply(lambda x:x.split())
df_txt.head(5)

,judul,tanggal,isi,kategori,cleaned_text,norm_text,token_text
0,Badan Kurus Tak Berarti Bebas dari Kolesterol ...,"Kamis, 05 Sep 2024 11:03 WIB",Jakarta - Tak sedikit yang beranggapan bahwa o...,Kesehatan,Jakarta Tak sedikit yang beranggapan bahwa ora...,jakarta tidak sedikit yang beranggapan bahwa o...,"[jakarta, tidak, sedikit, yang, beranggapan, b..."
1,Budaya Kerja Toksik di Jepang Picu 54 Karyawan...,"Kamis, 05 Sep 2024 10:38 WIB",Jakarta -\n \n Istilah 'bekerja ...,Kesehatan,Jakarta Istilah bekerja sampai mati nampaknya ...,jakarta istilah bekerja sampai mati nampaknya ...,"[jakarta, istilah, bekerja, sampai, mati, namp..."
2,Menyoal Serangan Jantung yang Dialami Faisal B...,"Kamis, 05 Sep 2024 10:03 WIB",Jakarta - Faisal Basri meninggal dunia. Ekonom...,Kesehatan,Jakarta Faisal Basri meninggal dunia Ekonom Un...,jakarta faisal basri meninggal dunia ekonom un...,"[jakarta, faisal, basri, meninggal, dunia, eko..."
3,Nyeri Kronis yang Bikin Paus Fransiskus Tak Bi...,"Kamis, 05 Sep 2024 09:37 WIB",Jakarta - Pemimpin tertinggi Gereja Katolik se...,Kesehatan,Jakarta Pemimpin tertinggi Gereja Katolik sedu...,jakarta pemimpin tertinggi gereja katolik sedu...,"[jakarta, pemimpin, tertinggi, gereja, katolik..."
4,"Perundungan PPDS Tak Cuma Terjadi di Undip, Be...","Kamis, 05 Sep 2024 08:35 WIB",Jakarta - Aksi bullying atau perundungan di ma...,Kesehatan,Jakarta Aksi bullying atau perundungan di masa...,jakarta aksi bullying atau perundungan di masa...,"[jakarta, aksi, bullying, atau, perundungan, d..."


##<b>2.4 Filtering/Stopwords Removal</b>

In [37]:
!pip install Sastrawi

In [39]:
import Sastrawi
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary

# Mendapatkan stop words bawaan dan menambahkannya
stop_words = StopWordRemoverFactory().get_stop_words()

# Membuat kamus baru untuk stop words
new_array = ArrayDictionary(stop_words)
stop_words_remover_new = StopWordRemover(new_array)

def stopword(tokens):
    """
    Menghapus stop words dari list of tokens (kata-kata).
    :param tokens: list of tokens (kata-kata)
    :return: list of tokens setelah stop words dihapus
    """
    if isinstance(tokens, list):  # Memastikan input adalah list
        # Gabungkan list menjadi string untuk diproses oleh Sastrawi
        text = ' '.join(tokens)
        # Hapus stopwords menggunakan Sastrawi
        filtered_text = stop_words_remover_new.remove(text)
        # Pisahkan kembali string menjadi list of tokens
        return filtered_text.split()
    else:
        raise ValueError("Input harus berupa list of tokens")

In [40]:
# Menggunakan fungsi text_remove_stopwords pada kolom 'token_text' dan menampilkan hasilnya di kolom baru 'swremove_text'
df_txt['swremove_text'] = df_txt['token_text'].apply(lambda x: stopword(x))
df_txt.head(5)

,judul,tanggal,isi,kategori,cleaned_text,norm_text,token_text,swremove_text
0,Badan Kurus Tak Berarti Bebas dari Kolesterol ...,"Kamis, 05 Sep 2024 11:03 WIB",Jakarta - Tak sedikit yang beranggapan bahwa o...,Kesehatan,Jakarta Tak sedikit yang beranggapan bahwa ora...,jakarta tidak sedikit yang beranggapan bahwa o...,"[jakarta, tidak, sedikit, yang, beranggapan, b...","[jakarta, sedikit, beranggapan, orang, berbada..."
1,Budaya Kerja Toksik di Jepang Picu 54 Karyawan...,"Kamis, 05 Sep 2024 10:38 WIB",Jakarta -\n \n Istilah 'bekerja ...,Kesehatan,Jakarta Istilah bekerja sampai mati nampaknya ...,jakarta istilah bekerja sampai mati nampaknya ...,"[jakarta, istilah, bekerja, sampai, mati, namp...","[jakarta, istilah, bekerja, mati, nampaknya, s..."
2,Menyoal Serangan Jantung yang Dialami Faisal B...,"Kamis, 05 Sep 2024 10:03 WIB",Jakarta - Faisal Basri meninggal dunia. Ekonom...,Kesehatan,Jakarta Faisal Basri meninggal dunia Ekonom Un...,jakarta faisal basri meninggal dunia ekonom un...,"[jakarta, faisal, basri, meninggal, dunia, eko...","[jakarta, faisal, basri, meninggal, dunia, eko..."
3,Nyeri Kronis yang Bikin Paus Fransiskus Tak Bi...,"Kamis, 05 Sep 2024 09:37 WIB",Jakarta - Pemimpin tertinggi Gereja Katolik se...,Kesehatan,Jakarta Pemimpin tertinggi Gereja Katolik sedu...,jakarta pemimpin tertinggi gereja katolik sedu...,"[jakarta, pemimpin, tertinggi, gereja, katolik...","[jakarta, pemimpin, tertinggi, gereja, katolik..."
4,"Perundungan PPDS Tak Cuma Terjadi di Undip, Be...","Kamis, 05 Sep 2024 08:35 WIB",Jakarta - Aksi bullying atau perundungan di ma...,Kesehatan,Jakarta Aksi bullying atau perundungan di masa...,jakarta aksi bullying atau perundungan di masa...,"[jakarta, aksi, bullying, atau, perundungan, d...","[jakarta, aksi, bullying, perundungan, masa, p..."


# Menyimpan hasil pre prosessing

In [41]:
# Menyimpan DataFrame ke file CSV
df_txt.to_csv('data_preprosessing.csv', index=False)

In [44]:
from google.colab import files

# Upload file  dari komputer
uploaded_preprosessing = files.upload()

Saving data_preprosessing.csv to data_preprosessing.csv


In [45]:
import pandas as pd
import io

# Membaca file CSV yang diunggah dan menyimpannya ke dalam DataFrame dengan variabel df_trans
df_pre = pd.read_csv(io.BytesIO(uploaded_preprosessing['data_preprosessing.csv']))
df_pre.head(5) # Menampilkan 5 baris pertama dari DataFrame

,judul,tanggal,isi,kategori,cleaned_text,norm_text,token_text,swremove_text
0,Badan Kurus Tak Berarti Bebas dari Kolesterol ...,"Kamis, 05 Sep 2024 11:03 WIB",Jakarta - Tak sedikit yang beranggapan bahwa o...,Kesehatan,Jakarta Tak sedikit yang beranggapan bahwa ora...,jakarta tidak sedikit yang beranggapan bahwa o...,"['jakarta', 'tidak', 'sedikit', 'yang', 'beran...","['jakarta', 'sedikit', 'beranggapan', 'orang',..."
1,Budaya Kerja Toksik di Jepang Picu 54 Karyawan...,"Kamis, 05 Sep 2024 10:38 WIB",Jakarta -\n \n Istilah 'bekerja ...,Kesehatan,Jakarta Istilah bekerja sampai mati nampaknya ...,jakarta istilah bekerja sampai mati nampaknya ...,"['jakarta', 'istilah', 'bekerja', 'sampai', 'm...","['jakarta', 'istilah', 'bekerja', 'mati', 'nam..."
2,Menyoal Serangan Jantung yang Dialami Faisal B...,"Kamis, 05 Sep 2024 10:03 WIB",Jakarta - Faisal Basri meninggal dunia. Ekonom...,Kesehatan,Jakarta Faisal Basri meninggal dunia Ekonom Un...,jakarta faisal basri meninggal dunia ekonom un...,"['jakarta', 'faisal', 'basri', 'meninggal', 'd...","['jakarta', 'faisal', 'basri', 'meninggal', 'd..."
3,Nyeri Kronis yang Bikin Paus Fransiskus Tak Bi...,"Kamis, 05 Sep 2024 09:37 WIB",Jakarta - Pemimpin tertinggi Gereja Katolik se...,Kesehatan,Jakarta Pemimpin tertinggi Gereja Katolik sedu...,jakarta pemimpin tertinggi gereja katolik sedu...,"['jakarta', 'pemimpin', 'tertinggi', 'gereja',...","['jakarta', 'pemimpin', 'tertinggi', 'gereja',..."
4,"Perundungan PPDS Tak Cuma Terjadi di Undip, Be...","Kamis, 05 Sep 2024 08:35 WIB",Jakarta - Aksi bullying atau perundungan di ma...,Kesehatan,Jakarta Aksi bullying atau perundungan di masa...,jakarta aksi bullying atau perundungan di masa...,"['jakarta', 'aksi', 'bullying', 'atau', 'perun...","['jakarta', 'aksi', 'bullying', 'perundungan',..."


In [46]:
# Merubah setiap elemen di kolom 'swremove_text' menjadi string
df_txt['swremove_text'] = df_txt['swremove_text'].astype(str)

In [47]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Menginisialisasi TfidfVectorizer
vectorizer = TfidfVectorizer()

# Menghitung TF-IDF
tfidf_matrix = vectorizer.fit_transform(df_txt['swremove_text'])

In [48]:
# Mengubah hasilnya menjadi DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())
tfidf_df.head(10)

,aachen,abad,abang,abc,abdalmasih,abidin,abinader,abror,absen,abuabu,...,zaitunalpukatselain,zaman,zat,zealand,zef,zenix,zero,zoonosis,zulies,zurich
0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.034334,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.071978,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.049785,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
